# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

09-29-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-09-30 04:23:09,33.93911,67.709953,199067,7799,NaN,NaN,Afghanistan,511.367798,3.917776
1,NaN,NaN,NaN,Albania,2022-09-30 04:23:09,41.15330,20.168300,332173,3589,NaN,NaN,Albania,11542.601988,1.080461
2,NaN,NaN,NaN,Algeria,2022-09-30 04:23:09,28.03390,1.659600,270668,6879,NaN,NaN,Algeria,617.244155,2.541490
3,NaN,NaN,NaN,Andorra,2022-09-30 04:23:09,42.50630,1.521800,46227,155,NaN,NaN,Andorra,59829.159387,0.335302
4,NaN,NaN,NaN,Angola,2022-09-30 04:23:09,-11.20270,17.873900,103131,1917,NaN,NaN,Angola,313.789810,1.858801


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/20/22,9/21/22,9/22/22,9/23/22,9/24/22,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,197608,197788,198023,198163,198244,198416,198543,198750,198876,199067
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,331715,331810,331861,331908,331953,331976,331987,332066,332129,332173
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,270612,270619,270625,270631,270637,270641,270649,270654,270662,270668


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/20/22,9/21/22,9/22/22,9/23/22,9/24/22,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7796,7796,7796,7796,7796,7796,7798,7798,7799,7799
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3589,3589,3589,3589,3589,3589,3589,3589,3589,3589
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6879,6879,6879,6879,6879,6879,6879,6879,6879,6879


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/20/22,9/21/22,9/22/22,9/23/22,9/24/22,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/20/22,9/21/22,9/22/22,9/23/22,9/24/22,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18303,18303,18359,18359,18359,18359,18359,18359,18359,18396
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,65353,65353,65544,65544,65544,65544,65544,65544,65544,65653
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,6873,6873,6881,6881,6881,6881,6881,6881,6881,6896


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/20/22,9/21/22,9/22/22,9/23/22,9/24/22,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,227,227,227,227,227,227,227,227,227,227
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,705,705,711,711,711,711,711,711,711,712
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,101,101,102,102,102,102,102,102,102,103


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
678,1001.0,Autauga,Alabama,US,2022-09-30 04:23:09,32.539527,-86.644082,18396,227,NaN,NaN,"Autauga, Alabama, US",32927.025721,1.233964
715,1075.0,Lamar,Alabama,US,2022-09-30 04:23:09,33.779950,-88.096680,4381,69,NaN,NaN,"Lamar, Alabama, US",31734.878667,1.574983
716,1077.0,Lauderdale,Alabama,US,2022-09-30 04:23:09,34.901719,-87.656247,28206,408,NaN,NaN,"Lauderdale, Alabama, US",30417.668690,1.446501


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,96346995,1059288,0.0
India,44587307,528629,0.0
France,35560024,156118,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,96346995,1059288,0.0,95287707.0,2022-09-30 04:23:09,37.934269,-91.444083
India,44587307,528629,0.0,44058678.0,2022-09-30 04:23:09,23.088275,81.806127
France,35560024,156118,0.0,35403906.0,2022-09-30 04:23:09,6.430808,-34.730285
Brazil,34654190,685927,0.0,33968263.0,2022-09-30 04:23:09,-12.669522,-48.480493
Germany,33312373,149948,0.0,33162425.0,2022-09-30 04:23:09,51.571844,10.277881


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,11260257,96023,0.0
Texas,7992596,90885,0.0
Florida,7150188,81369,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,11260257,96023,0.0,11164234.0,2022-09-30 04:23:09,37.843962,-120.728594
Texas,7992596,90885,0.0,7901711.0,2022-09-30 04:23:09,31.660643,-98.653069
Florida,7150188,81369,0.0,7068819.0,2022-09-30 04:23:09,28.940755,-82.700744
New York,6115959,71802,0.0,6044157.0,2022-09-30 04:23:09,42.544151,-75.474183
Illinois,3760119,39685,0.0,3720434.0,2022-09-30 04:23:09,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3454542,33598,0.0
Florida,Miami-Dade,1458095,11648,0.0
Arizona,Maricopa,1424974,17940,0.0
Illinois,Cook,1406047,14798,0.0
Texas,Harris,1214376,11306,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3454542,33598,0.0,3420944.0,2022-09-30 04:23:09,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1458095,11648,0.0,1446447.0,2022-09-30 04:23:09,25.611236,-80.551706,12086.0
Arizona,Maricopa,1424974,17940,0.0,1407034.0,2022-09-30 04:23:09,33.348359,-112.491815,4013.0
Illinois,Cook,1406047,14798,0.0,1391249.0,2022-09-30 04:23:09,41.841448,-87.816588,17031.0
Texas,Harris,1214376,11306,0.0,1203070.0,2022-09-30 04:23:09,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-27,198750,332066,270654,46147,103131,11,9089,9708420,442875,10207587,...,985422,244173,11950,544620,11475321,702768,535,11935,333503,257376
2022-09-28,198876,332129,270662,46227,103131,11,9089,9708420,442875,10209239,...,985422,244186,11950,544687,11476908,702768,535,11935,333503,257409
2022-09-29,199067,332173,270668,46227,103131,11,9098,9708420,442875,10238422,...,985422,244199,11950,544687,11477886,702768,535,11935,333531,257409


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-27,7798,3589,6879,155,1917,0,146,129897,8683,14963,...,7485,1637,14,5814,43147,5707,0,2157,4017,5599
2022-09-28,7799,3589,6879,155,1917,0,146,129897,8683,14984,...,7485,1637,14,5814,43147,5707,0,2157,4017,5602
2022-09-29,7799,3589,6879,155,1917,0,146,129897,8683,15171,...,7485,1637,14,5814,43148,5707,0,2157,4017,5602


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-09-28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-09-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/20/22,9/21/22,9/22/22,9/23/22,9/24/22,9/25/22,9/26/22,9/27/22,9/28/22,9/29/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18303,18303,18359,18359,18359,18359,18359,18359,18359,18396
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4350,4350,4375,4375,4375,4375,4375,4375,4375,4381
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,28080,28080,28147,28147,28147,28147,28147,28147,28147,28206
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,8815,8815,8843,8843,8843,8843,8843,8843,8843,8862
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,43969,43969,44084,44084,44084,44084,44084,44084,44084,44149


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-27,1517904,297562,8244,2268158,950441,11252371,1651763,894428,308037,49,...,2335380,7981482,1036535,141840,23148,2084569,1811170,599436,1863218,176728
2022-09-28,1517904,298178,8244,2271560,950940,11253703,1652689,895390,308252,49,...,2336504,7982208,1036535,142443,23154,2086021,1813501,599770,1864411,176728
2022-09-29,1522135,298178,8247,2271560,951283,11260257,1654124,896144,308452,49,...,2337590,7992596,1038416,142443,23162,2087360,1813501,600173,1865594,176728


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-09-27       18359   65544    6881  7504  16875    2797   6040   37492   
2022-09-28       18359   65544    6881  7504  16875    2797   6040   37492   
2022-09-29       18396   65653    6896  7526  16912    2802   6051   38211   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-09-27         9974     5991  ...    7436   2141     9251     2186   
2022-09-28         9974     5991  ...    7436   2141     9251     2186   
2022-09-29         9985     6008  ...    7436   2141     9251     2186   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-09-27          12143  11699  6145          0     2662   1814  
2022-09-28          12143  11699  6145          0     2662   1814  
2022-09-29          12143  11699  6145          0     2662   1814  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-27,20395,1364,34,31326,12106,95840,13288,11361,3104,0,...,27758,90825,5011,719,123,21808,14284,7402,15239,1894
2022-09-28,20395,1364,34,31370,12131,95869,13300,11365,3104,0,...,27766,90829,5011,719,123,21831,14312,7406,15256,1894
2022-09-29,20422,1364,34,31370,12148,96023,13307,11365,3104,0,...,27778,90885,5021,719,123,21849,14312,7406,15260,1894


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-09-27         227     711     102  107    258      54    129     655   
2022-09-28         227     711     102  107    258      54    129     655   
2022-09-29         227     712     103  107    258      54    129     656   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-09-27          168       87  ...     150     43       82       28   
2022-09-28          168       87  ...     150     43       82       28   
2022-09-29          169       87  ...     150     43       82       28   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-09-27            130    16    41          0       47     21  
2022-09-28            130    16    41          0       47     21  
2022-09-29            130    16    41          0       47     21  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-27,0.001043,0.000238,0.000018,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000160,...,0.0,0.000053,0.0,0.000163,0.000138,0.0,0.0,0.0,0.000192,0.000132
2022-09-28,0.000634,0.000190,0.000030,0.001734,0.0,0.0,0.00000,0.0,0.0,0.000162,...,0.0,0.000053,0.0,0.000123,0.000138,0.0,0.0,0.0,0.000000,0.000128
2022-09-29,0.000960,0.000132,0.000022,0.000000,0.0,0.0,0.00099,0.0,0.0,0.002858,...,0.0,0.000053,0.0,0.000000,0.000085,0.0,0.0,0.0,0.000084,0.000000


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-27,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.000870,...,0.0,0.0,0.0,0.0,0.000023,0.0,NaN,0.0,0.0,0.000000
2022-09-28,0.000128,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.001404,...,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,0.0,0.000536
2022-09-29,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.012480,...,0.0,0.0,0.0,0.0,0.000023,0.0,NaN,0.0,0.0,0.000000


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-27,0.000000,0.00000,0.000000,0.0000,0.000574,0.001457,0.000425,0.001006,0.000536,0.0,...,0.000000,0.000091,0.000000,0.000000,0.000432,0.000616,0.000000,0.000489,0.000538,0.002678
2022-09-28,0.000000,0.00207,0.000000,0.0015,0.000525,0.000118,0.000561,0.001076,0.000698,0.0,...,0.000481,0.000091,0.000000,0.004251,0.000259,0.000696,0.001287,0.000557,0.000640,0.000000
2022-09-29,0.002787,0.00000,0.000364,0.0000,0.000361,0.000582,0.000868,0.000842,0.000649,0.0,...,0.000465,0.001301,0.001815,0.000000,0.000345,0.000642,0.000000,0.000672,0.000634,0.000000


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                   \
Admin2           Autauga   Baldwin  Barbour      Bibb    Blount   Bullock   
2022-09-27      0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
2022-09-28      0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
2022-09-29      0.002015  0.001663  0.00218  0.002932  0.002193  0.001788   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-09-27      0.000000  0.000000  0.000000  0.000000  ...  0.002697   
2022-09-28      0.000000  0.000000  0.000000  0.000000  ...  0.000000   
2022-09-29      0.001821  0.019177  0.001103  0.002838  ...  0.000000   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-09-27      0.001872  0.003253  0.002752   0.000577  0.004378  0.003102   
2022-09-28      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2022-09-29      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-09-27            NaN  0.003771  0.000552  
2022-09-28            NaN  0.000000  0.000000  
2022-09-29            NaN  0.000000  0.000000  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-27,0.000000,0.0,0.0,0.000000,0.002152,0.000115,0.000452,0.001587,0.000967,NaN,...,0.000000,0.000033,0.000000,0.0,0.0,0.000964,0.00000,0.000811,0.000394,0.001587
2022-09-28,0.000000,0.0,0.0,0.001405,0.002065,0.000303,0.000903,0.000352,0.000000,NaN,...,0.000288,0.000044,0.000000,0.0,0.0,0.001055,0.00196,0.000540,0.001116,0.000000
2022-09-29,0.001324,0.0,0.0,0.000000,0.001401,0.001606,0.000526,0.000000,0.000000,NaN,...,0.000432,0.000616,0.001996,0.0,0.0,0.000825,0.00000,0.000000,0.000262,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                 \
Admin2         Autauga   Baldwin   Barbour Bibb Blount Bullock Butler   
2022-09-27         0.0  0.000000  0.000000  0.0    0.0     0.0    0.0   
2022-09-28         0.0  0.000000  0.000000  0.0    0.0     0.0    0.0   
2022-09-29         0.0  0.001406  0.009804  0.0    0.0     0.0    0.0   

Province_State                               ... Wyoming                  \
Admin2           Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2022-09-27      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   
2022-09-28      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   
2022-09-29      0.001527  0.005952      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-09-27          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2022-09-28          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2022-09-29          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-27,0.000870,0.000155,0.000021,0.000000,0.0,0.0,0.002248,0.000058,0.000820,0.000216,...,0.000325,0.000072,2.300000e-06,0.000133,0.000130,0.0,0.0,0.000008,0.000097,0.000097
2022-09-28,0.000752,0.000172,0.000025,0.000867,0.0,0.0,0.001124,0.000029,0.000410,0.000189,...,0.000163,0.000063,1.200000e-06,0.000128,0.000134,0.0,0.0,0.000004,0.000049,0.000113
2022-09-29,0.000856,0.000152,0.000024,0.000433,0.0,0.0,0.001057,0.000015,0.000205,0.001524,...,0.000081,0.000058,6.000000e-07,0.000064,0.000110,0.0,0.0,0.000002,0.000066,0.000056


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-27,0.000065,2.200000e-06,0.0,0.0,0.0,NaN,0.001724,0.000041,0.000117,0.000965,...,0.000404,0.0,0.0,2.000000e-06,0.000013,0.0,NaN,0.000015,0.0,0.000006
2022-09-28,0.000096,1.100000e-06,0.0,0.0,0.0,NaN,0.000862,0.000020,0.000059,0.001184,...,0.000202,0.0,0.0,1.000000e-06,0.000006,0.0,NaN,0.000007,0.0,0.000271
2022-09-29,0.000048,5.000000e-07,0.0,0.0,0.0,NaN,0.000431,0.000010,0.000029,0.006832,...,0.000101,0.0,0.0,5.000000e-07,0.000015,0.0,NaN,0.000004,0.0,0.000135


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-09-27,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-09-28,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-09-29,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-27,0.000060,0.000020,0.000051,0.000014,0.000411,0.000825,0.000423,0.000737,0.000811,0.0,...,0.000044,0.000287,0.000029,0.000031,0.000444,0.000722,0.000453,0.000675,0.000614,0.001350
2022-09-28,0.000030,0.001045,0.000026,0.000757,0.000468,0.000472,0.000492,0.000906,0.000754,0.0,...,0.000263,0.000189,0.000015,0.002141,0.000352,0.000709,0.000870,0.000616,0.000627,0.000675
2022-09-29,0.001409,0.000523,0.000195,0.000378,0.000414,0.000527,0.000680,0.000874,0.000702,0.0,...,0.000364,0.000745,0.000915,0.001071,0.000349,0.000676,0.000435,0.000644,0.000631,0.000338


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-09-27      0.000048  0.000046  0.000019  0.000049  0.000077  0.000051   
2022-09-28      0.000024  0.000023  0.000010  0.000024  0.000039  0.000025   
2022-09-29      0.001020  0.000843  0.001095  0.001478  0.001116  0.000906   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-09-27      0.000050  0.000306  0.000035  0.000061  ...  0.001361   
2022-09-28      0.000025  0.000153  0.000017  0.000030  ...  0.000680   
2022-09-29      0.000923  0.009665  0.000560  0.001434  ...  0.000340   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-09-27      0.000945  0.001644  0.001392   0.000295  0.002211  0.001566   
2022-09-28      0.000472  0.000822  0.000696   0.000147  0.001106  0.000783   
2022-09-29      0.000236  0.000411  0.000348   0.000074  0.000553  0.000392   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-09-27      -0.503937  0.001899  0.000278  
2022-09-28      -0.503937  0.000949  0.000139  
2022-09-29      -0.503937  0.000475  0.000069  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-09-27,0.000057,1.200000e-06,0.0,0.000021,0.001247,0.000177,0.000446,0.000798,0.000621,NaN,...,0.000063,0.000132,0.000031,0.000044,1.000000e-07,0.000743,0.000258,0.000898,0.000424,0.000799
2022-09-28,0.000028,6.000000e-07,0.0,0.000713,0.001656,0.000240,0.000675,0.000575,0.000310,NaN,...,0.000176,0.000088,0.000016,0.000022,1.000000e-07,0.000899,0.001109,0.000719,0.000770,0.000400
2022-09-29,0.000676,3.000000e-07,0.0,0.000356,0.001529,0.000923,0.000601,0.000287,0.000155,NaN,...,0.000304,0.000352,0.001006,0.000011,0.000000e+00,0.000862,0.000555,0.000360,0.000516,0.000200


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                             \
Admin2               Autauga   Baldwin   Barbour Bibb    Blount Bullock   
2022-09-27      6.000000e-07  0.000133  0.000155  0.0  0.000123     0.0   
2022-09-28      3.000000e-07  0.000067  0.000077  0.0  0.000062     0.0   
2022-09-29      1.000000e-07  0.000737  0.004941  0.0  0.000031     0.0   

Province_State                                      ...   Wyoming         \
Admin2         Butler   Calhoun  Chambers Cherokee  ...      Park Platte   
2022-09-27        0.0  0.000121  0.000094      0.0  ...  0.000026    0.0   
2022-09-28        0.0  0.000060  0.000047      0.0  ...  0.000013    0.0   
2022-09-29        0.0  0.000794  0.003000      0.0  ...  0.000007    0.0   

Province_State                                                              \
Admin2         Sheridan Sublette Sweetwater Teton         Uinta Unassigned   
2022-09-27          0.0      0.0        0.0   0.0  8.000000e-07       -1.0   
2022-09-28          0.0      0.0        0.0   0.0  4.000000e-07       -1.0   
2022-09-29          0.0      0.0        0.0   0.0  2.000000e-07       -1.0   

Province_State                  
Admin2         Washakie Weston  
2022-09-27          0.0    0.0  
2022-09-28          0.0    0.0  
2022-09-29          0.0    0.0  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 